Análisis de sentimientos de mensajes en Amazon usando Naive Bayes
===

El archivo que se encuentra disponible en el link

https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv

contiene mensajes escritos por los usuarios para productos comprados en Amazon y su valoración (positiva, negativa e indeterminada). En este laboratorio se debe construir un clasificador bayesiano que debe ser entrenado con los mensajes valorados, el cual debe ser posteriormente utilizado para valorar los mensajes con valoración indeterminada.

In [1]:
#
# Cargue el archivo usando Pandas e imprima la cantidad de
# registros
#
# Rta/
# 14609
#
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv", sep='\t', header=None)
df = df.rename(columns={0 : 'Texto', 1 : 'label'})
len(df)

14609

In [2]:
#
# Imprima el primer mensaje de texto.
#
# Rta/
# 'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'
#
df['Texto'][0]

'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'

In [3]:
#
# Imprima la cantidad de mensajes con NaN
#
# Rta/
# 13609
#
df['label'].isna().sum()

13609

In [4]:
#
# Imprima la cantidad de mensajes con valoración igual a 1.0
#
# Rta/
# 500
#
len(df[df['label'] == 1.0])

500

In [5]:
#
# Imprima la cantidad de mensajes con valoración igual a 0.0
#
# Rta/
# 500
#
len(df[df['label'] == 0.0])

500

In [6]:
#
# Genere un nuevo dataset que contenga únicamente los registros
# con valoración positiva o negativa e imprima su longitud
#
# Rta/
# 1000
#
db = df.dropna() 
len(df.dropna())

1000

In [7]:
#
# Genere una nueva columna en el nuevo dataset computada como
# el resultado de aplicar el stemmer de Porter al mensaje e
# imprima el primer mensaje transformado
#
# Rta/
# 'so there is no way for me to plug it in here in the us unless i go by a converter.'
#
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
db = db.rename(columns={0 : 'Texto', 1 : 'label'})
db['stemmed'] = db.Texto.apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()]))
db.iloc[0,2]

'so there is no way for me to plug it in here in the us unless i go by a converter.'

In [8]:
#
# Construya la matriz de terminos del documento considerando
# las palabras que tengan una frecuencia entre el 0.1% y el 98%,
# y que esten unicamente conformadas por letras.
#
# Imprima el tamaño del vocabulario.
#
# Rta/
# 1554
#
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(
    analyzer="word",                # a nivel de palabra
    lowercase=True,                 # convierte a minúsculas
    stop_words="english",           # stop_words en inglés
    token_pattern=r"(?u)\b\w\w+\b", # patrones a reconocer
    binary=False,                    # Los valores distintos de cero son fijados en 1
    max_df=0.98,                     # máxima frecuencia a considerar
    min_df=0.001,                       # ignora palabras con baja frecuencia
)

dtm = count_vect.fit_transform(db.stemmed)

dtm.shape[1]

1554

In [9]:
#
# Construya un clasificador bayesiano que use los primeros
# 500 patrones para entrenamiento y los últimos 500 para
# prueba, e imprima el porcentaje de datos para cada clase
# para la muestra de entrenamiento-
#
# Rta/
# 1.0    52.2
# 0.0    47.8
# Name: label, dtype: float64
#
X_train = dtm[0:500,]
X_test = dtm[500:,]

y_train_true = db.label[0:500]
y_test_true = db.label[500:]

round(100 * y_train_true.value_counts() / sum(y_train_true.value_counts()), 1)

1.0    52.2
0.0    47.8
Name: label, dtype: float64

In [10]:
#
# Imprima el porcentaje de datos para cada clase para la muestra
# de prueba, redondeado a un decimal.
#
# Rta/
# 0.0    52.2
# 1.0    47.8
# Name: label, dtype: float64
#
round(100 * y_test_true.value_counts() / sum(y_test_true.value_counts()), 1)

0.0    52.2
1.0    47.8
Name: label, dtype: float64

In [26]:
#
# Cree un clasificador de Bayes y entrenelo. Realice el pronostico
# para la muestra de entrenamiento y compute la matriz de confusion
#
# Rta/
# array([[214,  25],
#        [  1, 260]])
#
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

clf = BernoulliNB()
clf.fit(X_train.toarray(), y_train_true)
y_pred = clf.predict(X_train.toarray())
confusion_matrix(y_train_true, y_pred).astype('int')

array([[214,  25],
       [  1, 260]])

In [25]:
#
# Realice el pronóstico para la muestra de entrenamiento y compute
# la matriz de confusión
#
# Rta/
# array([[153, 108],
#        [ 32, 207]])
#
y_pred = clf.predict(X_test.toarray())
confusion_matrix(y_test_true, y_pred).astype('int')

array([[153, 108],
       [ 32, 207]])

In [ ]:
#
# Realice el pronostico para los mensajes con valoración 
# indeterminada y compute la cantidad de mensajes positivos
#
# Rta/
# 8284
#
df_Nan = df[df['label'].isnull()]
df_Nan = df_Nan.reset_index(drop=True)
df_Nan['stemmed'] = df_Nan.Texto.apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()])).str.lower()

X_Nan = count_vect.transform(df_Nan.stemmed)
# X_Nan.shape
pred = clf.predict(X_Nan)
Positivos = list(filter(lambda x: x==1,pred))
len(Positivos)

8284